<a href="https://colab.research.google.com/github/BrandonChiazza/economic-analysis-allowances/blob/main/Economic_Analysis_of_CA_Allowance_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#| default_exp elevate-climate-app

In [2]:
#| hide
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
from datetime import datetime

# The URL of the page with the CSV links
url = 'https://ww2.arb.ca.gov/our-work/programs/cap-and-trade-program/program-data/cap-and-trade-program-data-dashboard'

# Send a GET request to the page
response = requests.get(url)
print(response.status_code)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all the <a> tags that contain the CSV links
    csv_links = soup.find_all('a', href=lambda x: x and x.endswith('.csv'))
    print(csv_links)

    # Directory where you want to save the CSV files
    download_directory = "../data/"

    # Ensure the directory exists
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    # Get today's date in the format Month_Day_Year
    today_str = datetime.now().strftime("%B_%d_%Y")

    # Iterate over each link and download the CSV file
    for link in csv_links:
        # Get the absolute URL
        csv_url = urljoin(url, link['href'])

        # Make the request to download the file
        csv_response = requests.get(csv_url)

        # Extract the filename from the URL and sanitize it
        filename = os.path.basename(csv_url)
        filename = filename.replace("%20", " ")  # Replace URL-encoded spaces
        filename = filename.replace("/", "_")    # Replace any forward slashes which are not legal in filenames
        # Additional sanitization can be done here if necessary

        # Append today's date to the filename
        filename = f"{today_str}_{filename}"

        # Create the full path for the file
        file_path = os.path.join(download_directory, filename)

        # Save the file content to disk
        with open(file_path, 'wb') as csv_file:
            csv_file.write(csv_response.content)
        print(f'Downloaded: {file_path}')
else:
    print('Failed to retrieve the webpage')

200
[<a href="//ww2.arb.ca.gov/sites/default/files/2022-12/nc-allocation_v2023.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2022-12/nc-forest_buffer_balance.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2022-12/nc-HHI.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2022-12/nc-HHI.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2022-12/nc-qualified_bidders.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2022-12/nc-allowance_prices.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2022-12/nc-auction_proceeds.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2023-06/nc-market_transfers_by_year.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2023-06/nc-market_transfers_by_quarter.csv">Download chart data</a>, <a href="//ww2.arb.ca.gov/sites/default/files/2022-12/nc-VRE

In [3]:
"""# Install required libraries in Colab
!pip install pandas matplotlib seaborn ipywidgets

# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, FloatSlider
import requests
from io import StringIO

# Data cleaning and projection functions
def clean_price(price):
    #Convert price string to a float.
    return float(price.replace('$', '').replace(',', '').strip())

def project_future_prices(dataframe, increase_rate, inflation_rate, years=10):
    #Project future auction reserve prices based on increase rate and inflation rate.
    latest_price = dataframe['Auction Reserve Price'].iloc[-1]
    projected_years = [dataframe['Year'].iloc[-1] + i for i in range(1, years + 1)]
    projected_prices = [latest_price * ((1 + increase_rate) ** i) * ((1 + inflation_rate) ** i) for i in range(1, years + 1)]
    return pd.DataFrame({'Year': projected_years, 'Projected Auction Reserve Price': projected_prices})

# Visualization function with color customization, bands, and logo
def visualize_scenarios_with_bands(historical, projected, title_color, line_color, band_color, background_color):
    sns.set(style='white', rc={'axes.facecolor': background_color})
    fig, ax = plt.subplots(figsize=(14, 7))

    # Plot historical data
    ax.plot(historical['Year'], historical['Auction Reserve Price'], label='Historical', marker='o', color=line_color)

    # Plot projected data for base case
    ax.plot(projected['Year'], projected['Projected Auction Reserve Price'], label='Projected', linestyle='--', marker='x', color=line_color)

    # Create bands for upper and lower bounds
    upper_bound = [price * 1.05 for price in projected['Projected Auction Reserve Price']]
    lower_bound = [price * 0.95 for price in projected['Projected Auction Reserve Price']]

    # Fill between the upper and lower bounds
    ax.fill_between(projected['Year'], lower_bound, upper_bound, color=band_color, alpha=0.2)

    # Customize the title and labels
    plt.title('Historical and Projected Auction Reserve Prices', color=title_color)
    plt.xlabel('Year')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.show()

# Interactive visualization function
def interactive_visualization(increase_rate=0.05, inflation_rate=0.02):
    # Assuming the file is named 'November_09_2023_nc-allowance_prices.csv' and is uploaded to Colab
    file_path = 'November_09_2023_nc-allowance_prices.csv'
    allowance_prices = pd.read_csv(download_directory+"/"+file_path)

    # Data cleaning
    allowance_prices[['Quarter', 'Year']] = allowance_prices['Quarter Year'].str.split(' ', expand=True)
    allowance_prices['Year'] = allowance_prices['Year'].astype(int)
    allowance_prices['Auction Reserve Price'] = allowance_prices['Auction Reserve Price'].apply(clean_price)

    # Project future prices
    projected_prices_df = project_future_prices(allowance_prices, increase_rate, inflation_rate)

    # Define colors using provided RGB values converted to hex
    title_color = '#000b10'  # Dark shade of blue
    line_color = '#294646'   # Moderate greenish-gray
    band_color = '#2c4747'   # Medium gray with a slight purple tint
    background_color = '#f5f5f5'  # Off white

    # Visualize the scenarios with custom colors and bands
    visualize_scenarios_with_bands(allowance_prices, projected_prices_df, title_color, line_color, band_color, background_color)

# Set up the interactive widget
interact(interactive_visualization,
         increase_rate=FloatSlider(value=0.05, min=0, max=0.15, step=0.01, description='Increase Rate:'),
         inflation_rate=FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.01, description='Inflation Rate:'))

"""

'# Install required libraries in Colab\n!pip install pandas matplotlib seaborn ipywidgets\n\n# Import necessary libraries\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom ipywidgets import interact, FloatSlider\nimport requests\nfrom io import StringIO\n\n# Data cleaning and projection functions\ndef clean_price(price):\n    #Convert price string to a float.\n    return float(price.replace(\'$\', \'\').replace(\',\', \'\').strip())\n\ndef project_future_prices(dataframe, increase_rate, inflation_rate, years=10):\n    #Project future auction reserve prices based on increase rate and inflation rate.\n    latest_price = dataframe[\'Auction Reserve Price\'].iloc[-1]\n    projected_years = [dataframe[\'Year\'].iloc[-1] + i for i in range(1, years + 1)]\n    projected_prices = [latest_price * ((1 + increase_rate) ** i) * ((1 + inflation_rate) ** i) for i in range(1, years + 1)]\n    return pd.DataFrame({\'Year\': projected_years, \'Projected Auction Reserve 

In [4]:
#| hide
# This cell is hidden in the documentation, but it will execute in the notebook.
# Install required libraries
!pip install pandas matplotlib seaborn ipywidgets


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: C:\Users\brand\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
#| export
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, FloatSlider
import requests
from io import StringIO

In [6]:
#| export
# Data cleaning and projection functions
def clean_price(price):
    """Convert price string to a float."""
    return float(price.replace('$', '').replace(',', '').strip())

def project_future_prices(dataframe, increase_rate, inflation_rate, years=10):
    """Project future auction reserve prices based on increase rate and inflation rate."""
    latest_price = dataframe['Auction Reserve Price'].iloc[-1]
    projected_years = [dataframe['Year'].iloc[-1] + i for i in range(1, years + 1)]
    projected_prices = [latest_price * ((1 + increase_rate) ** i) * ((1 + inflation_rate) ** i) for i in range(1, years + 1)]
    return pd.DataFrame({'Year': projected_years, 'Projected Auction Reserve Price': projected_prices})

In [7]:
#| export
# Visualization function with color customization, bands, and logo
def visualize_scenarios_with_bands(historical, projected, title_color, line_color, band_color, background_color):
    sns.set(style='white', rc={'axes.facecolor': background_color})
    fig, ax = plt.subplots(figsize=(14, 7))

    # Plot historical data
    ax.plot(historical['Year'], historical['Auction Reserve Price'], label='Historical', marker='o', color=line_color)

    # Plot projected data for base case
    ax.plot(projected['Year'], projected['Projected Auction Reserve Price'], label='Projected', linestyle='--', marker='x', color=line_color)

    # Create bands for upper and lower bounds
    upper_bound = [price * 1.05 for price in projected['Projected Auction Reserve Price']]
    lower_bound = [price * 0.95 for price in projected['Projected Auction Reserve Price']]

    # Fill between the upper and lower bounds
    ax.fill_between(projected['Year'], lower_bound, upper_bound, color=band_color, alpha=0.2)

    # Customize the title and labels
    plt.title('Historical and Projected Auction Reserve Prices', color=title_color)
    plt.xlabel('Year')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.show()

# Interactive Visualization
Use the sliders below to adjust the increase rate and inflation rate, and see how the projected auction reserve prices change.

In [8]:
#| export
# Interactive visualization function
def interactive_visualization(increase_rate=0.05, inflation_rate=0.02):
    # Assuming the file is named 'November_09_2023_nc-allowance_prices.csv' and is uploaded to Colab
    file_path = 'November_09_2023_nc-allowance_prices.csv'
    allowance_prices = pd.read_csv(download_directory+'/'+file_path)

    # Data cleaning
    allowance_prices[['Quarter', 'Year']] = allowance_prices['Quarter Year'].str.split(' ', expand=True)
    allowance_prices['Year'] = allowance_prices['Year'].astype(int)
    allowance_prices['Auction Reserve Price'] = allowance_prices['Auction Reserve Price'].apply(clean_price)

    # Project future prices
    projected_prices_df = project_future_prices(allowance_prices, increase_rate, inflation_rate)

    # Define colors using provided RGB values converted to hex
    title_color = '#000b10'  # Dark shade of blue
    line_color = '#294646'   # Moderate greenish-gray
    band_color = '#2c4747'   # Medium gray with a slight purple tint
    background_color = '#f5f5f5'  # Off white

    # Visualize the scenarios with custom colors and bands
    visualize_scenarios_with_bands(allowance_prices, projected_prices_df, title_color, line_color, band_color, background_color)

In [9]:
#| export
# Set up the interactive widget
interact(interactive_visualization,
         increase_rate=FloatSlider(value=0.05, min=0, max=0.15, step=0.01, description='Increase Rate:'),
         inflation_rate=FloatSlider(value=0.02, min=-0.05, max=0.1, step=0.01, description='Inflation Rate:'))

interactive(children=(FloatSlider(value=0.05, description='Increase Rate:', max=0.15, step=0.01), FloatSlider(…

<function __main__.interactive_visualization(increase_rate=0.05, inflation_rate=0.02)>

In [10]:
#!nbdev_build_lib

from nbdev import *
#!nbdev_build_docs
!nbdev_export



!nbdev_clean_nbs
!nbdev_build_lib
!nbdev_test_nbs
!nbdev_build_docs
!git pull
!git add -A
!git commit -m update
!git push

#updated

'nbdev_clean_nbs' is not recognized as an internal or external command,
operable program or batch file.
'nbdev_build_lib' is not recognized as an internal or external command,
operable program or batch file.
'nbdev_test_nbs' is not recognized as an internal or external command,
operable program or batch file.
'nbdev_build_docs' is not recognized as an internal or external command,
operable program or batch file.


Already up to date.


The file will have its original line endings in your working directory


[main 82dd517] update
 11 files changed, 487 insertions(+), 149 deletions(-)
 create mode 100644 data/November_10_2023_nc-HHI.csv
 create mode 100644 data/November_10_2023_nc-VREP_remaining.csv
 create mode 100644 data/November_10_2023_nc-VREP_retirements.csv
 create mode 100644 data/November_10_2023_nc-allocation_v2023.csv
 create mode 100644 data/November_10_2023_nc-allowance_prices.csv
 create mode 100644 data/November_10_2023_nc-auction_proceeds.csv
 create mode 100644 data/November_10_2023_nc-forest_buffer_balance.csv
 create mode 100644 data/November_10_2023_nc-market_transfers_by_quarter.csv
 create mode 100644 data/November_10_2023_nc-market_transfers_by_year.csv
 create mode 100644 data/November_10_2023_nc-qualified_bidders.csv


To https://github.com/BrandonChiazza/elevate-climate.git
   8be191c..82dd517  main -> main
